In [1]:
from graphMatching import *
from networkx import read_edgelist
from scipy.io import loadmat
from model import *
from utils import *

In [2]:
# import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
from matplotlib.patches import ConnectionPatch

In [3]:
def test_matching(truth, test):
    matching = []
    test = test.tolist()
    truth = truth.tolist()

    for item in test:
        if item in truth:
            matching.append(item)
    return matching

In [4]:
data = "ACM_DBLP" # args.dataset

device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")
train_features = {}
if (data == "ACM_DBLP"):
    train_set = ["ACM", "DBLP"]
    input_dim = 17
    b = np.load('data/ACM-DBLP.npz')
    train_features["ACM"] = [torch.from_numpy(b["x1"]).float()]
    train_features["DBLP"] = [torch.from_numpy(b["x2"]).float()]
    test_pairs = b['test_pairs'].astype(np.int32)
    NUM_HIDDEN_LAYERS = 12
    HIDDEN_DIM = 1024
    output_feature_size = 1024
    lr = 0.0001
    epoch = 100
elif (data == "Douban Online_Offline"):
    a1, f1, a2, f2, test_pairs = load_douban()
    f1 = f1.A
    f2 = f2.A
    train_set = ["Online", "Offline"]
    input_dim = 538
    test_pairs = torch.tensor(np.array(test_pairs, dtype=int)) - 1
    test_pairs = test_pairs.numpy()
    train_features["Online"] = [torch.from_numpy(f1).float()]
    train_features["Offline"] = [torch.from_numpy(f2).float()]
    NUM_HIDDEN_LAYERS = 6
    HIDDEN_DIM = 512
    output_feature_size = 512
    lr = 0.0001
    epoch = 100


In [5]:
""" temp = torch.from_numpy(b["x1"]).float() # G1, features
print(temp.shape)
temp """

' temp = torch.from_numpy(b["x1"]).float() # G1, features\nprint(temp.shape)\ntemp '

In [6]:
""" temp = torch.from_numpy(b["x2"]).float() # G2, features
print(temp.shape)
temp """

' temp = torch.from_numpy(b["x2"]).float() # G2, features\nprint(temp.shape)\ntemp '

In [7]:
""" temp = b["test_pairs"]
print(temp.shape)
temp """

' temp = b["test_pairs"]\nprint(temp.shape)\ntemp '

In [8]:
test_pairs

array([[   0, 6829],
       [   2, 3102],
       [   3, 3584],
       ...,
       [9841, 3392],
       [9850,  306],
       [9868, 9011]], dtype=int32)

In [9]:
train_set

['ACM', 'DBLP']

In [10]:
encoder = "GIN"
use_input_augmentation = True
use_output_augmentation = False
print("Loading training datasets")
train_loader = {}
for dataset in train_set:
    train_loader[dataset] = [load_adj(dataset)]

train_loader

Loading training datasets


{'ACM': [tensor([[0., 1., 1.,  ..., 0., 0., 0.],
          [1., 0., 1.,  ..., 0., 0., 0.],
          [1., 1., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]])],
 'DBLP': [tensor([[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]])]}

In [11]:
train_loader.keys()

dict_keys(['ACM', 'DBLP'])

In [12]:
""" temp = train_loader["ACM"][0]
print(temp.shape)
temp """

' temp = train_loader["ACM"][0]\nprint(temp.shape)\ntemp '

In [13]:
""" temp = train_loader["DBLP"][0]
print(temp.shape)
temp """

' temp = train_loader["DBLP"][0]\nprint(temp.shape)\ntemp '

In [14]:
model = GAE(NUM_HIDDEN_LAYERS,
            input_dim,
            HIDDEN_DIM,
            output_feature_size, activation=F.relu,
            use_input_augmentation=use_input_augmentation,
            use_output_augmentation=use_output_augmentation,
            encoder=encoder).to(device)
model

GAE(
  (base_gcn): GIN(
    (in_proj): Linear(in_features=17, out_features=1024, bias=True)
    (convs): ModuleList(
      (0-13): 14 x GINConv(
        (linear): Linear(in_features=1041, out_features=1024, bias=True)
      )
    )
    (out_proj): Linear(in_features=15360, out_features=1024, bias=True)
  )
)

In [15]:
print("Generating training features")


Generating training features


In [16]:
print("Fitting model")
# fit_GAE_real(data, len(train_set) * (1 + 1), model, epoch, train_loader, train_features, device, lr,test_pairs)
# fit_GAE_real(data, no_samples, GAE, epoch, train_loader, train_features, device, lr, test_pairs):

no_samples = len(train_set) * (1 + 1)
GAE = model
# ---

best_hitAtOne = 0
best_hitAtFive = 0
best_hitAtTen = 0
best_hitAtFifty = 0
optimizer = Adam(GAE.parameters(), lr=lr, weight_decay=5e-4)

for step in tqdm(range(epoch)):
    loss = 0
    
    for dataset in train_loader.keys():
        S = train_loader[dataset][0]
        initial_features = train_features[dataset]
        
        for i in range(len(train_loader[dataset])):
            adj_tensor = train_loader[dataset][i]
            adj = coo_matrix(adj_tensor.numpy())
            adj_norm = preprocess_graph(adj)
            pos_weight = float(adj.shape[0] * adj.shape[0] - adj.sum()) / adj.sum()
            norm = adj.shape[0] * adj.shape[0] / float((adj.shape[0] * adj.shape[0] - adj.sum()) * 2)

            adj_label = coo_matrix(S.numpy())
            adj_label = sparse_to_tuple(adj_label)

            adj_norm = torch.sparse.FloatTensor(torch.LongTensor(adj_norm[0].T),
                                                torch.FloatTensor(adj_norm[1]),
                                                torch.Size(adj_norm[2])).to(device)
            adj_label = torch.sparse.FloatTensor(torch.LongTensor(adj_label[0].T),
                                                torch.FloatTensor(adj_label[1]),
                                                torch.Size(adj_label[2])).to(device)

            initial_feature = initial_features[i].to(device)

            weight_mask = adj_label.to_dense().view(-1) == 1
            weight_tensor = torch.ones(weight_mask.size(0))
            weight_tensor[weight_mask] = pos_weight
            weight_tensor = weight_tensor.to(device)
            z = GAE(initial_feature, adj_norm)
            A_pred = torch.sigmoid(torch.matmul(z,z.t()))
            loss += norm * F.binary_cross_entropy(A_pred.view(-1), adj_label.to_dense().view(-1),
                                                        weight=weight_tensor)
    
    optimizer.zero_grad()
    loss = loss / no_samples
    loss.backward()
    optimizer.step()

    # ---
    keys = list(train_loader.keys())
    S1 = train_loader[keys[0]][0]
    S2 = train_loader[keys[1]][0]
    
    adj_S1 = coo_matrix(S1.numpy())
    adj_norm_1 = preprocess_graph(adj_S1)
    adj_norm_1 = torch.sparse.FloatTensor(torch.LongTensor(adj_norm_1[0].T),
                                            torch.FloatTensor(adj_norm_1[1]),
                                            torch.Size(adj_norm_1[2])).to(device)
    adj_S2 = coo_matrix(S2.numpy())
    adj_norm_2 = preprocess_graph(adj_S2)
    adj_norm_2 = torch.sparse.FloatTensor(torch.LongTensor(adj_norm_2[0].T),
                                            torch.FloatTensor(adj_norm_2[1]),
                                            torch.Size(adj_norm_2[2])).to(device)
    if (data == "ACM_DBLP"):
        S1_feat = train_features["ACM"][0]
        S2_feat = train_features["DBLP"][0]
    elif (data == "Douban Online_Offline"):
        S1_feat = train_features["Online"][0]
        S2_feat = train_features["Offline"][0]

    # ---
    S1_emb = GAE(S1_feat.to(device), adj_norm_1).detach()
    S2_emb = GAE(S2_feat.to(device), adj_norm_2).detach()

    D = torch.cdist(S1_emb, S2_emb, 2) # Euclidean distance
    
    if (data == "ACM_DBLP"):
        test_idx = test_pairs[:, 0].astype(np.int32)
        labels = test_pairs[:, 1].astype(np.int32)
    elif (data == "Douban Online_Offline"):
        test_idx = test_pairs[0, :].astype(np.int32)
        labels = test_pairs[1, :].astype(np.int32)
    
    hitAtOne = 0
    hitAtFive = 0
    hitAtTen = 0
    hitAtFifty = 0
    hitAtHundred = 0
    
    # test
    
    for i in range(len(test_idx)): # here
        dist_list = D[test_idx[i]]
        # print(i, test_idx[i], dist_list)
        sorted_neighbors = torch.argsort(dist_list).cpu()
        label = labels[i]
        
        """ if i == 0:
            print(label, sorted_neighbors[0].item(), sorted_neighbors)
            # 6829 6829 tensor([6829, 3102,  601,  ..., 7878, 9701, 2044]) """
        
        for j in range(100):
            if (sorted_neighbors[j].item() == label):
                if (j == 0):
                    hitAtOne += 1
                    hitAtFive += 1
                    hitAtTen += 1
                    hitAtFifty += 1
                    hitAtHundred += 1
                    break
                elif (j <= 4):
                    hitAtFive += 1
                    hitAtTen += 1
                    hitAtFifty += 1
                    hitAtHundred += 1
                    break
                elif (j <= 9):
                    hitAtTen += 1
                    hitAtFifty += 1
                    hitAtHundred += 1
                    break
                elif (j <= 49):
                    hitAtFifty += 1
                    hitAtHundred += 1
                    break
                elif (j <= 100):
                    hitAtHundred += 1
                    break
    
    cur_hitAtOne = hitAtOne / len(test_idx)
    cur_hitAtFive = hitAtFive / len(test_idx)
    cur_hitAtTen = hitAtTen / len(test_idx)
    cur_hitAtFifty = hitAtFifty / len(test_idx)

    if(cur_hitAtOne > best_hitAtOne): best_hitAtOne = cur_hitAtOne
    if (cur_hitAtFive > best_hitAtFive): best_hitAtFive = cur_hitAtFive
    if (cur_hitAtTen > best_hitAtTen): best_hitAtTen = cur_hitAtTen
    if (cur_hitAtFifty > best_hitAtFifty): best_hitAtFifty = cur_hitAtFifty

print("The best results achieved:")
print("Hit@1: ", end="")
print(best_hitAtOne)
print("Hit@5: ", end="")
print(best_hitAtFive)
print("Hit@10: ", end="")
print(best_hitAtTen)
print("Hit@50: ", end="")
print(best_hitAtFifty)

Fitting model


100%|██████████| 100/100 [05:41<00:00,  3.42s/it]

The best results achieved:
Hit@1: 0.7383399209486166
Hit@5: 0.9142292490118578
Hit@10: 0.950790513833992
Hit@50: 0.9814229249011858


In [17]:
D.shape

torch.Size([9872, 9916])

In [18]:
D

tensor([[17.4764, 18.5462, 17.2963,  ..., 17.6960, 17.3824, 17.1963],
        [15.8282, 14.8613, 15.4932,  ..., 15.9868, 15.6872, 15.5217],
        [21.4269, 22.2469, 21.3279,  ..., 21.5871, 21.3105, 21.2061],
        ...,
        [ 2.2839,  4.6655,  1.1718,  ...,  3.6780,  2.2081,  1.9188],
        [ 2.0562,  4.7448,  1.9882,  ...,  3.5978,  1.7922,  0.4190],
        [ 1.8103,  4.8147,  1.8327,  ...,  3.5856,  1.4176,  1.3181]],
       device='cuda:0')

Truth

In [19]:
print(data)
print(test_pairs.shape)
print(test_pairs)

ACM_DBLP
(5060, 2)
[[   0 6829]
 [   2 3102]
 [   3 3584]
 ...
 [9841 3392]
 [9850  306]
 [9868 9011]]


In [20]:
if (data == "ACM_DBLP"):
    test_pairs_ = test_pairs
elif (data == "Douban Online_Offline"):
    test_pairs_ = test_pairs.T
    
truth = test_pairs_[test_pairs_[:, 1].argsort()]
print(truth.shape)
truth

(5060, 2)


array([[3615,    1],
       [1302,    2],
       [ 466,    3],
       ...,
       [6148, 9907],
       [3826, 9911],
       [7275, 9915]], dtype=int32)

Option 0

In [21]:
def hungarian(D):
    print("0")
    P = torch.zeros_like(D)
    matrix = D.tolist()
    m = Munkres()
    print("1")
    indexes = m.compute(matrix)
    print("2")
    total = 0
    for r, c in tqdm(indexes):
        print(r)
        P[r][c] = 1
        total += matrix[r][c]
    return P.t()

P = hungarian(D)
print(P.shape)
# online_offline: 215m
# acm_dblp: 1112m
P

0
1


In [ ]:
# save

X = P.cpu().numpy()
np.save("P_hungarian_ACM_DBLP", X)

In [ ]:
# load

P = np.load("P_hungarian.npy")
P

In [ ]:
# from Hungarian (source code)
option0 = []
m, n = P.shape
for i in range(m):
    for j in range(n):
        if P[i][j] == 1:
            option0.append([j, i]) # S, S_hat
option0 = np.array(option0)
print(len(option0))
option0

In [ ]:
matching = test_matching(truth, option0)
print(len(matching))
print(len(matching) / len(truth))
matching

Option 1

In [ ]:
import numpy as np
from scipy.optimize import linear_sum_assignment

def hungarian_algorithm(cost_matrix):
    # Use the linear_sum_assignment method from scipy
    row_ind, col_ind = linear_sum_assignment(cost_matrix)
    
    # Total cost
    total_cost = cost_matrix[row_ind, col_ind].sum()
    
    # The assignments are returned as (row, col) pairs
    assignments = list(zip(row_ind, col_ind))
    
    return total_cost, assignments

total_cost, assignments = hungarian_algorithm(D.cpu())

In [ ]:
option1 = np.array(assignments)
option1 = option1[option1[:, 1].argsort()]
print(option1.shape)
option1

In [ ]:
matching = test_matching(truth, option1)
print(len(matching))
print(len(matching) / len(truth))
matching

In [ ]:
# build graph
def build_graph(adj_norm):
    edges = adj_norm.coalesce().indices()

    G = nx.from_edgelist(edges.T.cpu().numpy())
    G.remove_edges_from(nx.selfloop_edges(G))
    print(G.number_of_nodes(), G.number_of_edges())

    return G

G1 = build_graph(adj_norm_1)
G2 = build_graph(adj_norm_2)

In [ ]:
pos1 = nx.spring_layout(G1)
pos2 = nx.spring_layout(G2)

In [ ]:
plt.figure(figsize=(16, 8))
ax1 = plt.subplot(1, 2, 1)
plt.title('Graph 1')
nx.draw_networkx(G1, pos=pos1, font_color="w")
ax2 = plt.subplot(1, 2, 2)
plt.title('Graph 2')
nx.draw_networkx(G2, pos=pos2, font_color="w")

# add connections
for i in range(len(matching)):
    con = ConnectionPatch(xyA=pos1[matching[i][0]], xyB=pos2[matching[i][1]], coordsA=ax1.transData, coordsB=ax2.transData, arrowstyle="-", color="green")
    ax2.add_artist(con)
plt.show()

Option 2

In [ ]:
import pygmtools as pygm

X = pygm.hungarian(D.cpu().numpy())
print(X.shape)
X

In [ ]:
indices = []
row, col = X.shape
for i in range(row):
    for j in range(col):
        if X[i][j] == 1:
            indices.append([i, j])
option2 = np.array(indices)
option2 = option2[option2[:, 1].argsort()]
print(option2.shape)
option2

In [ ]:
matching = test_matching(truth, option2)
print(len(matching))
print(len(matching) / len(truth))
matching

Option 3

In [ ]:
indices = []
for i in range(D.shape[0]):
    dist_list = D[i]
    sorted_neighbors = torch.argsort(dist_list).cpu()
    indices.append([i, sorted_neighbors[0]])

option3 = np.array(indices)
option3 = option3[option3[:, 1].argsort()]
print(option3.shape)
option3

In [ ]:
matching = test_matching(truth, option3)
print(len(matching))
print(len(matching) / len(truth))
matching

---

In [ ]:
a = torch.tensor([[0.0,  0.0], [0.0, 1.0], [0.0,  2.0]])
print(a)
b = torch.tensor([[0.0, 1.0 ], [1.0,  1.0]])
print(b)
torch.cdist(a, b, p=2)

In [ ]:
a = torch.tensor([[4.01, 3.0, 2.0, 0.1, 4.0]])
print(a)
torch.argsort(a, dim=1)

In [ ]:
a = np.array([[9, 2, 3],
              [4, 5, 6],
              [7, 0, 5]])

a[a[:, 2].argsort()]
